# NumPy-like Operations

@[Xiaoyu Chen](mailto:c-xy17@tsinghua.org.cn)
@[Chaoming Wang](https://github.com/chaoming0625)

## `brainpy.math.array` and `brainpy.math.Variable`

Neural modeling often involves the computation of thousands or even millions of elements and requires large-scale computation. To support this, BrainPy provides a basic data structure `brainpy.math.array` with abundant operations. In fact, the concept of `brainpy.math.array` is the same as `ndarray` in NumPy, and the operations used in BrainPy closely resemble NumPy operations. 

```{note}
Users who are not familiar with the Python package NumPy may refer to [https://numpy.org/](https://numpy.org/) for more information.
```

Due to JIT compilation, however, once a array is given to the JIT compiler, the values inside the array cannot be changed. This gives rise to severe limitations because some dynamically changing variables like membrane potentials cannot be stored in `brainpy.math.array`. To extend arrays to store changing values, BrainPy offers a new data structure named ``brainpy.math.Variable``.

Simply speaking, ``brainpy.math.Variable`` is a pointer referring to an array. If an array is wrapped into a Variable, its value can change over time. For more information about ``brainpy.math.Variable``, please refer to [`Variable` and `BrainPbject`](variables.ipynb).

## NumPy-like operations for arrays and variables

For `brainpy.math.array`, BrainPy provides array-related operations that have the same form as NumPy while capable of performing JIT transformations. Almost all NumPy operations can be used in BrainPy as long as users transform `numpy.xxx` to `brainpy.math.xxx`.

Specifically, the NumPy-like operations for `brainpy.math.array` include:

1. Creating an array: users can use `bm.array()`， `bm.ones()`， `bm.zeros()`， `bm.arange()`， and so on to create an array.
2. Element-wise operations: `+`, `-`, `*`, `/`, ...
3. Aggregation functions: `.max()`, `.min()`, `.sum()`, `.mean()`, `.prod()`, ...
4. Broadcasting: operating two arrays that have different dimensions (usually a smaller array is broadcast to a larger array with compatible shapes to complete operation).
5. Indexing, slicing, and iterating: they are the same operations done on sequence data structures such as lists.
6. Other mathematical and logical functions: `bm.sin()`, `bm.cos()`, `bm.sort()`, `bm.argmax()`, `bm.where()`, ...

In [31]:
import brainpy.math as bm

bm.set_platform('cpu')

Here are some examples of array-related operations in BrainPy:

In [32]:
# to create an array
a1 = bm.array([[0, 1, 2], [3, 4, 5]])
a1

Array(value=DeviceArray([[0, 1, 2],
                         [3, 4, 5]]),
      dtype=int32)

In [33]:
# element-wise operation
a1 += 1
a1

Array(value=DeviceArray([[1, 2, 3],
                         [4, 5, 6]]),
      dtype=int32)

In [34]:
a2 = bm.ones(3)
a2

Array(value=DeviceArray([1., 1., 1.]), dtype=float32)

In [35]:
# slicing
a2[0:2] = 10
a2

Array(value=DeviceArray([10., 10.,  1.]), dtype=float32)

In [36]:
# broadcasting
a3 = a1 + a2
a3

Array(value=DeviceArray([[11., 12.,  4.],
                         [14., 15.,  7.]]),
      dtype=float32)

In [37]:
# sorting
bm.sort(a3)

Array(value=DeviceArray([[ 4., 11., 12.],
                         [ 7., 14., 15.]]),
      dtype=float32)

In [38]:
# reshaping
a4 = a.reshape(6, -1)
a4

Array(value=DeviceArray([[1],
                         [2],
                         [3],
                         [4],
                         [5],
                         [6]]),
      dtype=int32)

For `brainpy.math.Variable`, since the data inside a Variable is an array, many operations on arrays can be used on Variables as well. Nevertheless, there are some requirements for updating a Variable. For example, directly changing the value of a Variable will return an array but not a Variable:

In [39]:
v1 = bm.Variable(a1)
v1

Variable(value=DeviceArray([[1, 2, 3],
                            [4, 5, 6]]),
         dtype=int32)

In [40]:
# Directly changing the value of a Variable will return an array but not a Variable
v1 = v1 + 1
v1

Array(value=DeviceArray([[2, 3, 4],
                         [5, 6, 7]]),
      dtype=int32)

Though the value of the variable has changed in this example, similar operations will fail on JIT compilation. This is because the JIT compiler does not allow an array to be assigned with a new value. To update a Variable, users need to use **in-place updating**. Detailed instructions are contained in [`Variable` and `BrainPbject`](variables.ipynb).

## When to use `brainpy.math.array` and `brainpy.math.Variable`?

Whether to use `brainpy.math.array` or `brainpy.math.Variable` depends on whether the values inside the array change when the code is running. Common neural features like the membrane potential and the gating variables of voltage-dependent ion channels should be defined as Variables, while static properties such as the membrane capacitance, the voltage threshold, and static synaptic connections should be defined as arrays (or a unified single value).